# i. Perkenalan

# ii. Import Libraries

In [148]:
# Import Statistics Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import chi2_contingency, pointbiserialr

# Sklearn Libraries for Classification ML
from sklearn.feature_selection import f_classif
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score, KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, RobustScaler, StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, precision_score, recall_score, accuracy_score, roc_curve, roc_auc_score

from xgboost import XGBClassifier

from tensorflow.keras.metrics import Recall
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

import time

# Save and Load Libraries
import pickle

# Import Warnings
import warnings
warnings.filterwarnings('ignore')

# iii. Data Loading

In [149]:
# Load data untuk dataset EDA
df = pd.read_csv('dataset/P2_FinPro_Final_Record.csv')
df

,id,gender,car,realty,child_number,income,occupation,education,marital_status,house_type,...,mobile_phone,work_phone,phone,email,family_size,months_balance,status,begin_month,credit_approval,target
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,...,1,1,0,0,2.0,0,C,-15,No,1
1,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,...,1,1,0,0,2.0,-1,C,-15,No,1
2,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,...,1,1,0,0,2.0,-2,C,-15,No,1
3,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,...,1,1,0,0,2.0,-3,C,-15,No,1
4,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,...,1,1,0,0,2.0,-4,C,-15,No,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
777710,5150337,M,N,Y,0,112500.0,Working,Secondary / secondary special,Single / not married,Rented apartment,...,1,0,0,0,1.0,-9,0,-13,No,1
777711,5150337,M,N,Y,0,112500.0,Working,Secondary / secondary special,Single / not married,Rented apartment,...,1,0,0,0,1.0,-10,2,-13,No,1
777712,5150337,M,N,Y,0,112500.0,Working,Secondary / secondary special,Single / not married,Rented apartment,...,1,0,0,0,1.0,-11,1,-13,No,1
777713,5150337,M,N,Y,0,112500.0,Working,Secondary / secondary special,Single / not married,Rented apartment,...,1,0,0,0,1.0,-12,0,-13,No,1


## Mengecek ukuran dataset

In [150]:
print(f'Datase ini memiliki {df.shape[0]} baris dan {df.shape[1]} kolom')

Datase ini memiliki 777715 baris dan 22 kolom


## Mengecek informasi dataset

In [151]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 777715 entries, 0 to 777714
Data columns (total 22 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   id               777715 non-null  int64  
 1   gender           777715 non-null  object 
 2   car              777715 non-null  object 
 3   realty           777715 non-null  object 
 4   child_number     777715 non-null  int64  
 5   income           777715 non-null  float64
 6   occupation       777715 non-null  object 
 7   education        777715 non-null  object 
 8   marital_status   777715 non-null  object 
 9   house_type       777715 non-null  object 
 10  days_birth       777715 non-null  int64  
 11  days_employed    777715 non-null  int64  
 12  mobile_phone     777715 non-null  int64  
 13  work_phone       777715 non-null  int64  
 14  phone            777715 non-null  int64  
 15  email            777715 non-null  int64  
 16  family_size      777715 non-null  floa

## Mengecek statistik sederhana dataset

In [152]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
id,777715.0,5.078743e+06,41804.424817,5008804.0,5044568.5,5069530.0,5115551.0,5150487.0
child_number,777715.0,4.280823e-01,0.745755,0.0,0.0,0.0,1.0,19.0
income,777715.0,1.885348e+05,101622.450077,27000.0,121500.0,162000.0,225000.0,1575000.0
days_birth,777715.0,-1.612494e+04,4104.304018,-25152.0,-19453.0,-15760.0,-12716.0,-7489.0
days_employed,777715.0,5.777583e+04,136471.735391,-15713.0,-3292.0,-1682.0,-431.0,365243.0
mobile_phone,777715.0,1.000000e+00,0.000000,1.0,1.0,1.0,1.0,1.0
work_phone,777715.0,2.318176e-01,0.421993,0.0,0.0,0.0,0.0,1.0
phone,777715.0,3.009650e-01,0.458678,0.0,0.0,0.0,1.0,1.0
email,777715.0,9.167497e-02,0.288567,0.0,0.0,0.0,0.0,1.0
family_size,777715.0,2.208837e+00,0.907380,1.0,2.0,2.0,3.0,20.0


## Mengecek missing value

In [153]:
df.isna().sum()

id                 0
gender             0
car                0
realty             0
child_number       0
income             0
occupation         0
education          0
marital_status     0
house_type         0
days_birth         0
days_employed      0
mobile_phone       0
work_phone         0
phone              0
email              0
family_size        0
months_balance     0
status             0
begin_month        0
credit_approval    0
target             0
dtype: int64

## Mengecek data duplikat

In [154]:
print(f'Terdapat {df.duplicated().sum()} data duplikat')

Terdapat 0 data duplikat


# v. Feature Engineering

## Feature Creation

#### Age

Mengubah days_birth menjadi age

In [155]:
df['age'] = (-df['days_birth'] // 365)


#### Years of Employment

In [156]:
df['years_employed'] = df['days_employed'].apply(lambda x: 0 if x > 0 else int(-x // 365))

#### Education Income Ratio

In [157]:
edu_order = {
    'Lower secondary': 1,
    'Secondary / secondary special': 2,
    'Incomplete higher': 3,
    'Higher education': 4,
    'Academic degree': 5
}

df['education_ordinal'] = df['education'].map(edu_order)

In [158]:
df['education_income_ratio'] = df['education_ordinal'] / df['income']

### Income per Person

Formula: income / family_size

Reason:

A person with a high income but many dependents may still struggle financially. This feature captures the financial capacity per individual in the household, which is more informative than just total income

Relationship to credit approval:

The higher the income per person, the better the repayment capacity, increasing the likelihood of credit approval

In [159]:
df['income_per_person'] = df['income'] / df['family_size'].replace(0, 1)

### has child ?

Formula: child_number > 0 (Boolean 1/0)

Reason:

Having children generally increases financial responsibilities and monthly expenses.

Relationship to credit approval:

Applicants with children might be viewed as higher risk, especially if their income is not significantly high, because their financial obligations are greater

In [160]:
df['has_child'] = (df['child_number'] > 0).astype(int)

### low income ?

Formula:
1 if income < median_income, else 0

Reason:

Applicants below median income are considered to have relatively limited financial resources compared to the population

Relationship to Credit Approval:

Lower income correlates with reduced repayment ability, leading to higher perceived risk for lenders

In [161]:
median_income = df['income'].median()

In [162]:
df['low_income'] = (df['income'] < median_income).astype(int)

### Has child and low income ?

Formula:
1 if has_child == 1 and income < median_income, else 0

Reason:

Applicants who have children and earn below the median income are likely to face higher financial burdens with limited financial capacity

Relationship to Credit Approval:

Applicants with both children and low income are typically considered higher risk, making credit approval less likely

In [163]:
df['has_child_and_low_income'] = ((df['has_child'] == 1) & (df['income'] < median_income)).astype(int)

### Financial Dependence Ratio

Formula:

family_size / (1 + child_number)<br/>
(Avoid division by zero; if no children, denominator = 1)

Reason:

This captures the proportion of adults vs dependents in the household. A lower ratio suggests fewer adults supporting more dependents

In [164]:
df['financial_dependence_ratio'] = df['family_size']/(1 + df['child_number'])

### Is Single ?

Formula: marital_status == 'Single' (Boolean 1/0)

Reason:

Marital status influences financial burdens and spending patterns. Single individuals often have more predictable expenses and fewer obligations

Relationship to credit approval:

Single applicants are sometimes seen as lower risk due to simpler financial circumstances, potentially improving approval chances

In [165]:
df['is_single'] = df['marital_status'].isin(['Single / not married', 'Separated', 'Widow']).astype(int)

In [166]:
df.columns

Index(['id', 'gender', 'car', 'realty', 'child_number', 'income', 'occupation',
       'education', 'marital_status', 'house_type', 'days_birth',
       'days_employed', 'mobile_phone', 'work_phone', 'phone', 'email',
       'family_size', 'months_balance', 'status', 'begin_month',
       'credit_approval', 'target', 'age', 'years_employed',
       'education_ordinal', 'education_income_ratio', 'income_per_person',
       'has_child', 'low_income', 'has_child_and_low_income',
       'financial_dependence_ratio', 'is_single'],
      dtype='object')

In [167]:
cat_cols = ['gender', 'car', 'realty', 'occupation', 'marital_status', 'house_type']
num_cols = ['child_number', 'family_size', 'months_balance', 'age', 'years_employed', 'income', 'education_income_ratio', 'income_per_person', 'financial_dependence_ratio']
exclude_cols = ['work_phone', 'email', 'phone', 'has_child', 'low_income', 'has_child_and_low_income', 'is_single', 'education_ordinal']

num_cols = [col for col in num_cols if col not in exclude_cols]

print(num_cols)
print(cat_cols)
print(exclude_cols)

['child_number', 'family_size', 'months_balance', 'age', 'years_employed', 'income', 'education_income_ratio', 'income_per_person', 'financial_dependence_ratio']
['gender', 'car', 'realty', 'occupation', 'marital_status', 'house_type']
['work_phone', 'email', 'phone', 'has_child', 'low_income', 'has_child_and_low_income', 'is_single', 'education_ordinal']


Fitur yang dihapus karena sudah digantikan oleh fitur lain

In [168]:
df = df.drop(columns=['id', 'education', 'days_birth', 'days_employed', 'begin_month', 'target', 'status', 'mobile_phone'])

Mengubah fitur credit_approval (target) menjadi binary

In [169]:
df['credit_approval'] = df['credit_approval'].map({'Yes': 1, 'No':0})

## Handling Cardinality

In [170]:
# Set display untuk filter kolom nantinya
pd.set_option('display.max_colwidth', None)

listCols = []
for col in cat_cols:
    listCols.append([col, df[col].nunique(), df[col].unique()])


pd.DataFrame(columns=['Nama Kolom', 'Jumlah Nilai Unique', 'Nilai Unique'], data=listCols)

,Nama Kolom,Jumlah Nilai Unique,Nilai Unique
0,gender,2,"[M, F]"
1,car,2,"[Y, N]"
2,realty,2,"[Y, N]"
3,occupation,5,"[Working, Commercial associate, Pensioner, State servant, Student]"
4,marital_status,5,"[Civil marriage, Married, Single / not married, Separated, Widow]"
5,house_type,6,"[Rented apartment, House / apartment, Municipal apartment, With parents, Co-op apartment, Office apartment]"


## Split fitur dan target

In [171]:
# Splitting antara X dan Y
X = df.drop(['credit_approval'], axis=1) # Features
y = df['credit_approval'] # Target

In [172]:
X.head()

,gender,car,realty,child_number,income,occupation,marital_status,house_type,work_phone,phone,...,age,years_employed,education_ordinal,education_income_ratio,income_per_person,has_child,low_income,has_child_and_low_income,financial_dependence_ratio,is_single
0,M,Y,Y,0,427500.0,Working,Civil marriage,Rented apartment,1,0,...,32,12,4,0.000009,213750.0,0,0,0,2.0,0
1,M,Y,Y,0,427500.0,Working,Civil marriage,Rented apartment,1,0,...,32,12,4,0.000009,213750.0,0,0,0,2.0,0
2,M,Y,Y,0,427500.0,Working,Civil marriage,Rented apartment,1,0,...,32,12,4,0.000009,213750.0,0,0,0,2.0,0
3,M,Y,Y,0,427500.0,Working,Civil marriage,Rented apartment,1,0,...,32,12,4,0.000009,213750.0,0,0,0,2.0,0
4,M,Y,Y,0,427500.0,Working,Civil marriage,Rented apartment,1,0,...,32,12,4,0.000009,213750.0,0,0,0,2.0,0


In [173]:
y.describe()

count    777715.000000
mean          0.859113
std           0.347905
min           0.000000
25%           1.000000
50%           1.000000
75%           1.000000
max           1.000000
Name: credit_approval, dtype: float64

## Split train set dan test set

In [174]:
# Splitting antara train & test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42, stratify=y)
print('Train Size', X_train.shape)
print('Test Size', X_test.shape)

Train Size (622172, 23)
Test Size (155543, 23)


## Penangan Data Outlier

In [175]:
# Display skewness distributions and outliers

# Buat dataframe skewness dahulu dari kolom numerik
listCols2= []
for col in num_cols:
  listCols2.append([col, round(X_train[col].skew(),1), np.where(
    (round(X_train[col].skew(),2) <= 0.5) & (round(X_train[col].skew(),2) >= -0.5),
    'normal','skewed')])

skewness = pd.DataFrame(columns=['Nama Kolom', 'Nilai Skewness', 'Distribution'], data= listCols2)
skewness

# Buat dataframe persentase outliers dahulu dari kolom numerik
column = []
lower_bound = []
upper_bound = []
Percentage_Outliers = []
for row in range (0, len(skewness)):
  col = skewness['Nama Kolom'][row]
  #checking upper and lower boundary
  if skewness['Distribution'][row] == 'Skewed':
    IQR = X_train[col].quantile(0.75) - X_train[col].quantile(0.25)
    lower_boundary = X_train[col].quantile(0.25) - (IQR * 3)
    upper_boundary = X_train[col].quantile(0.75) + (IQR * 3)
  else:
    lower_boundary = X_train[col].mean() - 3* X_train[col].std()
    upper_boundary = X_train[col].mean() + 3* X_train[col].std()
  #append to list
  column.append(col)
  lower_bound.append(lower_boundary)
  upper_bound.append(upper_boundary)
  totout = ((len(X_train[X_train[col] > upper_boundary]) / len(X_train) * 100) + (len(X_train[X_train[col] < lower_boundary]) / len(X_train) * 100))
  Percentage_Outliers.append(totout)


outliers = pd.DataFrame({
    'upper_boundary': [round(upper_bound,2) for upper_bound in upper_bound],
    'lower_boundary': [round(lower_bound,2) for lower_bound in lower_bound],
    'Percentage Outliers': [round(Percentage_Outliers,2) for Percentage_Outliers in Percentage_Outliers],   
})

# Concat and Merge the two
frames = [skewness, outliers]
Skewness_Outliers = pd.concat(frames, ignore_index=False, axis=1)
Skewness_Outliers

,Nama Kolom,Nilai Skewness,Distribution,upper_boundary,lower_boundary,Percentage Outliers
0,child_number,2.5,skewed,2.66,-1.81,1.43
1,family_size,1.3,skewed,4.93,-0.51,1.38
2,months_balance,-0.6,skewed,22.86,-61.60,0.00
3,age,0.2,normal,77.42,9.94,0.00
4,years_employed,1.7,skewed,25.79,-13.89,2.03
5,income,2.5,skewed,493247.40,-116208.25,1.41
6,education_income_ratio,2.3,skewed,0.00,-0.00,1.30
7,income_per_person,2.7,skewed,316340.27,-115524.00,1.51
8,financial_dependence_ratio,-0.5,normal,2.87,0.39,0.01


In [176]:
num_cols_normal = ['normal', 'age']
num_cols_skewed = ['child_number',
                    'family_size',
                    'months_balance',
                    'years_employed',
                    'education_income_ratio',
                    'income_per_person']

# Model Training - Phase 1

In [177]:
# Definisikan pipeline untuk numerikal kolom dataset 2
num_transformer_normal = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=5)),
    ('scaler', StandardScaler())
])

num_transformer_skewed = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=5)),
    ('scaler', RobustScaler())
])

# Definisikan pipeline untuk kategorikal kolom dataset 2
cat_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

# Pipeline untuk preprocessing dataset 2
preprocessing_pipeline = ColumnTransformer(transformers=[
    ('numerical_normal', num_transformer_normal, num_cols_normal),
    ('numerical_skewed', num_transformer_skewed, num_cols_skewed),
    ('categorical', cat_transformer, cat_cols),
    ('num_passthrough', 'passthrough', exclude_cols)
])

In [179]:
preprocessing_pipeline

ColumnTransformer(transformers=[('numerical_normal',
                                 Pipeline(steps=[('imputer', KNNImputer()),
                                                 ('scaler', StandardScaler())]),
                                 ['normal', 'age']),
                                ('numerical_skewed',
                                 Pipeline(steps=[('imputer', KNNImputer()),
                                                 ('scaler', RobustScaler())]),
                                 ['child_number', 'family_size',
                                  'months_balance', 'years_employed',
                                  'education_income_ratio',
                                  'income_per_person']),
                                ('categorical',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('encoder',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['gender', 'car', 'realty', 'occupation',
                                  'marital_status', 'house_type']),
                                ('num_passthrough', 'passthrough',
                                 ['work_phone', 'email', 'phone', 'has_child',
                                  'low_income', 'has_child_and_low_income',
                                  'is_single', 'education_ordinal'])])

In [25]:
# Pipeline dengan KNN Classifier
pipeknn = Pipeline([
    ('preprocessor', preprocessing_pipeline),
    ('knn', KNeighborsClassifier())
])

# Pipeline dengan Decision Tree Classifier
pipedt = Pipeline([
    ('preprocessor', preprocessing_pipeline),
    ('decisiontree', DecisionTreeClassifier())
])

# Pipeline dengan Random Forest Classifier
piperf= Pipeline([
    ('preprocessor', preprocessing_pipeline),
    ('randomforest', RandomForestClassifier())
])

# Pipeline dengan SVM Classifier
pipesvm = Pipeline([
    ('preprocessor', preprocessing_pipeline),
    ('svm', SVC())
])

# Pipeline dengan XGBoost Classifier
pipexgb = Pipeline([
    ('preprocessor', preprocessing_pipeline),
    ('xgb', XGBClassifier())
])

# **Melatih model**
pipeknn.fit(X_train, y_train)
pipedt.fit(X_train, y_train)
piperf.fit(X_train, y_train)
# pipesvm.fit(X_train, y_train)
pipexgb.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numerical_normal',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['mobile_phone', 'age']),
                                                 ('numerical_skewed',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer()),
                                                                  ('scaler',
                                                                   RobustScaler())]),
                                                  ['child_number',
                                                   'family_size',
                                                   'months_balance',
                                                   'years_employed',
                                                   'education_income_...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=None, n_jobs=None,
                               num_parallel_tree=None, random_state=None, ...))])

In [ ]:
def ann_model():
    model = Sequential()
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # Binary
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=[Recall()])
    return model

ann_classifier = KerasClassifier(model=ann_model, epochs=50, batch_size=32, verbose=0)

In [27]:
pipeann = Pipeline(steps=[
    ('preprocessor', preprocessing_pipeline),
    ('classifier', ann_classifier)
])

# Model Evaluation - Phase 1

In [28]:
# Simpan hasil evaluasi dalam list
eval_results = []

# Evaluasi model klasifikasi dengan metrik Recall (Train & Test) dan waktu eksekusi
for model_name, model in [("KNN Classifier", pipeknn),
                          ("Decision Tree Classifier", pipedt),
                          ("Random Forest Classifier", piperf),
                          ("XGB Classifier", pipexgb),
                          ("ANN Classifier", pipeann)]:

    start_time = time.time()

    # Fit setiap model (bukan hanya pipeann!)
    start_fit = time.time()
    model.fit(X_train, y_train)
    fit_time = time.time() - start_fit

    # Prediksi data train dan test
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)

    # Hitung Recall
    recall_train = recall_score(y_train, y_pred_train, average='binary')
    recall_test = recall_score(y_test, y_pred_test, average='binary')

    exec_time = time.time() - start_time  # Hitung waktu eksekusi

    # Simpan hasil evaluasi dalam list
    eval_results.append([model_name, recall_train, recall_test, exec_time])

# Tampilkan hasil dalam bentuk tabel
eval_df = pd.DataFrame(eval_results, columns=["Model", "Recall Train", "Recall Test", "Exec Time (s)"])
print(eval_df)


ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense" is incompatible with the layer: expected axis -1 of input shape to have value 18, but received input with shape (None, 40)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(None, 40), dtype=float32)
  • training=True
  • mask=None
  • kwargs=<class 'inspect._empty'>

In [ ]:
# Initialization for KFold = 5
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Model and Classifier list (menggunakan pipeline)
classifiers = [pipeknn, pipedt, piperf, pipexgb, pipeann]
model_names = ['KNN', 'Decision Tree', 'Random Forest', 'XGBoost', 'ANN']

# Create a list for the results
cv_results = []

# Loop evaluasi dengan scoring recall
for name, model in zip(model_names, classifiers):
    scores_recall = cross_val_score(model, X_train, y_train, scoring='recall', cv=cv)
    
    cv_results.append({
        'Model': name,
        'CV Recall (Mean)': round(scores_recall.mean(), 4),
        'CV Recall (Stdev)': round(scores_recall.std(), 4),
    })

# Buat dataframe hasil
df_cv_eval = pd.DataFrame(cv_results)
print(df_cv_eval)